In [1]:

from enum import Enum
import numpy as np


# Implementação algoritmica

### Tipos de validações

In [2]:
class CriterioGaussJacobi(Enum):
    SASSENFELD = 'Critério de Sassenfeld'
    LINHAS = 'Critério das linhas'
    

### Validações

In [3]:

def validarCriterio(X: np.matrix, criterio: list[CriterioGaussJacobi] | CriterioGaussJacobi = None) -> bool:
    
    resultado = {}
    n, m = X.shape

    # Uses all
    if not criterio:
        criterio = [_ for _ in CriterioGaussJacobi]

    # Make sure it is a list
    if not isinstance(criterio, list):
        criterio = [criterio]


    if CriterioGaussJacobi.SASSENFELD in criterio:
        
        betas = np.ones(n)

        for i in range(n):
            soma = np.sum([abs(X[i, j]) * betas[j] for j in range(m) if i != j])
            betas[i] = soma / X[i, i]

        resultado[CriterioGaussJacobi.SASSENFELD.value] = np.max(betas) < 1
    
    if CriterioGaussJacobi.LINHAS in criterio:
        
        alphas = np.array([abs(X[i, j]) for j in range(m) for i in range(n) if i == j])
        
        for i in range(n):
            soma = np.sum([abs(X[i, j]) for j in range(m) if i != j])
            alphas[i] /= soma

        resultado[CriterioGaussJacobi.LINHAS.value] = np.max(alphas) < 1

    return resultado


### Teste das validações

In [4]:

A = np.matrix([[1, 0.5, -0.1, 0.1], 
                [0.2, 1, -0.2, -0.1], 
                [-0.1, -0.2, 1, 0.2], 
                [0.1, 0.3, 0.2, 1]])

validarCriterio(A)

{'Critério de Sassenfeld': True, 'Critério das linhas': False}

### Método de Gauss-Seidel

In [5]:

def metodoGaussSeidel(A, b, startPoint, tol = 10e-5, maxiter = 1000):

    n, m = A.shape
    x_act = np.float64(np.copy(startPoint))

    for iter in range(maxiter):
        x_ant = np.copy(x_act)

        for i in range(n):
            sumKnown = 0 
            sumPrevi = 0

            for j in range(m):
                if j < i:
                    sumKnown += A[i, j] * x_act[j]
                elif j > i:
                    sumPrevi += A[i, j] * x_ant[j]
            
            x_act[i] = (b[i] - sumKnown - sumPrevi) / A[i, i]

        print(f'Iteration {iter + 1}: {x_act}')

        if np.max(np.abs(x_act - x_ant)) / np.max(np.abs(x_act)) <= tol:
            print(f'Converged in {iter + 1} iterations')
            return x_act
        
    print('Did not converge')
    print(x_act)

    return
    


### Teste do método com exercício do livro (Exemplo 14)

In [6]:

X = np.matrix([[5.0, 1.0, 1.0], 
               [3.0, 4.0, 1.0], 
               [3.0, 3.0, 6.0]])

b = np.array([5.0, 6.0, 0.0])

metodoGaussSeidel(X, b, np.array([0,0,0]), tol=(5 * 10e-2), maxiter=5)

Iteration 1: [ 1.     0.75  -0.875]
Iteration 2: [ 1.025   0.95   -0.9875]
Converged in 2 iterations


array([ 1.025 ,  0.95  , -0.9875])

#  Exercicio 22

In [7]:

A1 = np.matrix([[10.0, 1.0, 1.0], 
                [1.0, 10.0, 1.0], 
                [1.0, 1.0, 10.0]])

b1 = np.array([12.0, 12.0, 12.0])

A2 = np.matrix([[4.0, -1.0, 0.0, 0.0],
               [-1.0, 4.0, -1.0, 0.0], 
               [0.0, -1.0, 4.0, -1.0], 
               [0.0, 0.0, -1.0, 4.0]])

b2 = np.array([1.0, 1.0, 1.0, 1.0])


### 22.a) Verifique se o critério de sassenfeld é satisfeito

In [8]:
# Para a matriz A1
validarCriterio(A1, CriterioGaussJacobi.SASSENFELD)

{'Critério de Sassenfeld': True}

In [9]:
# Para a matriz A2
validarCriterio(A2, CriterioGaussJacobi.SASSENFELD)

{'Critério de Sassenfeld': True}

### 22.b) Resolva por Gauss-Seidel, se possível

In [10]:

A1_params = {
    'tol': 10e-5,
    'maxiter': 1000,
    'startPoint': np.array([2, 2, 2])
}

metodoGaussSeidel(A1, b1, **A1_params)


Iteration 1: [0.8   0.92  1.028]
Iteration 2: [1.0052   0.99668  0.999812]
Iteration 3: [1.0003508  0.99998372 0.99996655]
Iteration 4: [1.00000497 1.00000285 0.99999922]
Iteration 5: [0.99999979 1.0000001  1.00000001]
Converged in 5 iterations


array([0.99999979, 1.0000001 , 1.00000001])

In [11]:

A2_params = {
    'tol': 10e-5,
    'maxiter': 1000,
    'startPoint': np.array([2, 2, 2, 2])
}

metodoGaussSeidel(A2, b2, **A2_params)


Iteration 1: [0.75       0.9375     0.984375   0.49609375]
Iteration 2: [0.484375   0.6171875  0.52832031 0.38208008]
Iteration 3: [0.40429688 0.4831543  0.46630859 0.36657715]
Iteration 4: [0.37078857 0.45927429 0.45646286 0.36411572]
Iteration 5: [0.36481857 0.45532036 0.45485902 0.36371475]
Iteration 6: [0.36383009 0.45467228 0.45459676 0.36364919]
Iteration 7: [0.36366807 0.45456621 0.45455385 0.36363846]
Iteration 8: [0.36364155 0.45454885 0.45454683 0.36363671]
Converged in 8 iterations


array([0.36364155, 0.45454885, 0.45454683, 0.36363671])

## Exercício 23

### 23.a) [...]

Resolvido à mão, anexo.

### 23.b) Escolha o menor valor positivo de k e faça duas iterações do método de Gauss Seidel para o sistema obtido.

In [12]:

# 4,0000000001
k = 4 + 10e-10

X = np.matrix([[k, 3, 1], 
              [k, 6, 1], 
              [1, 6, 7]])

y = np.array([1, 2, 3])

metodoGaussSeidel(X, y, startPoint=np.array([0, 0, 0]), maxiter=2)

Iteration 1: [0.25       0.16666667 0.25      ]
Iteration 2: [0.0625     0.25       0.20535714]
Did not converge
[0.0625     0.25       0.20535714]


### 23.c) Comente o erro cometido no item (b)

O menor valor possível para K deveria ser o limite de k tendendo à 4 por valores maiores do que ele. Enquanto a implementação foi realizada com o valor de $4 + 0.1 \times 10e^{-10}$.